In [1]:
import configparser
import boto3
import pandas as pd

# Goal 4
Create a config file and read the credentials and regions from there.

In [2]:
path=""
def load_credentials():
    config = configparser.RawConfigParser()
    config.read(path+"config")
    config.read(path+"credentials")
    
    AWS_KEY_ID = config.get('default', 'aws_access_key_id')
    AWS_SECRET = config.get('default', 'aws_secret_access_key')
    REGION = config.get('default', 'region')
    
    return AWS_KEY_ID,AWS_SECRET,REGION

AWS_KEY_ID,AWS_SECRET,REGION=load_credentials()

In [3]:
# initalize client
ec2 = boto3.resource('ec2',aws_access_key_id=AWS_KEY_ID,
                          aws_secret_access_key=AWS_SECRET,
                          region_name=REGION)

client = boto3.client('ec2',
                      region_name=REGION,
                      aws_access_key_id=AWS_KEY_ID,
                      aws_secret_access_key=AWS_SECRET)


# Goal 1
Question this command must answer: What EC2 instances are running|stopped|shutdown?

In [4]:
def get_ec2_state(ec2,value):
    
    ec2 = boto3.resource('ec2',aws_access_key_id=AWS_KEY_ID,
                          aws_secret_access_key=AWS_SECRET,
                          region_name=REGION)
    #  can us this list as well ['pending','running','shutting-down','terminated','stopping','stopped']
    if value == "all":
        value = ['running','stopped','terminated']
    else:
        value = [value]
        
    instances = ec2.instances.filter(Filters=[{'Name': 'instance-state-name', 'Values':value}])
    
    instanceId=[]
    instanceName=[]
    
    for instance in instances:
        instanceId.append(instance.id)
        instanceName.append(instance.state['Name'])
        
    data={'Instance':instanceId,'State':instanceName}
    df=pd.DataFrame(data)
    
    if len(df)>0:
        dataframe=df
    else:
        dataframe="No Instances"
    
    return dataframe

df_1=get_ec2_state(ec2,"all")

df_1.head()

,Instance,State
0,i-0b0ac919e7caa75a6,running
1,i-0d05abb7c4717a3cc,stopped


In [5]:
def ec2_monitoring_status(ec2,value):
    
    if value == "all":
        value = ['enabled','disabled']
    else:
        value = [value]
        
    instances = ec2.instances.filter(
    Filters=[{'Name': 'monitoring-state', 'Values':value}])
    instanceId=[]
    instanceMontr=[]
    
    for instance in instances:
        instanceId.append(instance.id)
        instanceMontr.append(instance.monitoring['State'])
        
    data={'Instance':instanceId,'Monitoring':instanceMontr}
    df=pd.DataFrame(data)

    if len(df)>0:
        dataframe=df
    else:
        dataframe="No Instances"
    
    return dataframe
df_2=ec2_monitoring_status(ec2,"all")

df_2.head()

,Instance,Monitoring
0,i-0b0ac919e7caa75a6,disabled
1,i-0d05abb7c4717a3cc,disabled


# Goal 3
Question this command must answer: What EC2 instances are in this region?

In [9]:
def get_ec2_list(ec2):
    
    instanceId=[]
    instanceMontr=[]
    instacestate=[]
    for instance in ec2.instances.all():
        
        instanceId.append(instance.id)
        instanceMontr.append(instance.monitoring['State'])
        instacestate.append(instance.state['Name'])
        
    df = pd.DataFrame({"Instance_ID":instanceId,"Monitoring":instanceMontr,"State":instacestate})
    
    if len(df)>0:
        dataframe=df
    else:
        dataframe="No Instances"
    
    return dataframe 

df_3=get_ec2_list(ec2)

df_3.head()

,Instance_ID,Monitoring,State
0,i-0b0ac919e7caa75a6,disabled,running
1,i-0d05abb7c4717a3cc,disabled,stopped


# Goal 5
Question: How many EBS volumes are not in use?

In [7]:
def ebs_volume_unused(client):
    
    volumes_not_used = list()

    volume_detail = client.describe_volumes()

    # process each volume in volume_detail
    if volume_detail['ResponseMetadata']['HTTPStatusCode'] == 200:
        for each_volume in volume_detail['Volumes']:
            # some logging to make things clear about the volumes in your existing system
            print("Working for volume with volume_id: ", each_volume['VolumeId'])
            print("State of volume: ", each_volume['State'])
            print("Attachment state length: ", len(each_volume['Attachments']))
            print(each_volume['Attachments'])
            print("--------------------------------------------")
            # figuring out the unused volumes
            # the volumes which do not have 'Attachments' key and their state is 'available' is considered to be unused
            if len(each_volume['Attachments']) == 0 and each_volume['State'] == 'available':
                volumes_not_used.append(each_volume['VolumeId'])
                
    return volumes_not_used

unused_volume=ebs_volume_unused(client)
print(unused_volume)

Working for volume with volume_id:  vol-03944e7763bfb1146
State of volume:  in-use
Attachment state length:  1
[{'AttachTime': datetime.datetime(2021, 10, 28, 11, 12, 58, tzinfo=tzutc()), 'Device': '/dev/xvda', 'InstanceId': 'i-0d05abb7c4717a3cc', 'State': 'attached', 'VolumeId': 'vol-03944e7763bfb1146', 'DeleteOnTermination': True}]
--------------------------------------------
Working for volume with volume_id:  vol-095e31efe67b08a1c
State of volume:  in-use
Attachment state length:  1
[{'AttachTime': datetime.datetime(2021, 10, 28, 11, 15, 1, tzinfo=tzutc()), 'Device': '/dev/xvda', 'InstanceId': 'i-0b0ac919e7caa75a6', 'State': 'attached', 'VolumeId': 'vol-095e31efe67b08a1c', 'DeleteOnTermination': True}]
--------------------------------------------
[]


In [8]:
def ebs_volume_unencrypted(client):
    
    not_encrypted=[]
    test =[]
    response = client.describe_volumes()
    volumelist = response['Volumes']
    if type(volumelist) is not list:
        test.append(volumelist)
    else:
        test.extend(volumelist)
    for vol in volumelist:
        if vol['Encrypted'] == False:
            print("VolumeId :"+vol['State']+""+vol['State'])
            not_encrypted.append(vol['VolumeId'])
    return not_encrypted

unecncrypted_volume=ebs_volume_unencrypted(client)
print(unecncrypted_volume)

VolumeId :in-usein-use
VolumeId :in-usein-use
['vol-03944e7763bfb1146', 'vol-095e31efe67b08a1c']
